## Sentiment Analysis: Sampling & Labeling

1. 상품 카테고리 별 리뷰 비율을 최대한 반영하여 총 2천 문장의 학습 표본을 추출합니다.

2. 표본에서 학습에 활용할 ground truth label을 생성합니다.

    * `gpt-4.1`을 활용하여 LLM-based Few-shot Learning을 수행합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os
import time

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [ ]:
class Sentiment(BaseModel):
    sentiment: str = Field(description="분석된 감성. '긍정', '부정', '중립' 중 하나의 값이어야 합니다.")

In [ ]:
template = """
당신은 한국 화장품 리뷰의 감성을 분석하는 최고의 전문가입니다.
주어진 문장을 분석하여 '긍정', '부정', '중립' 중 하나로 분류해주세요.

# 감성 분류 기준:
- "긍정": 제품에 대한 만족, 칭찬, 추천, 재구매 의사 등 긍정적인 내용.
- "부정": 제품에 대한 불만, 단점, 비판, 부작용 등 부정적인 내용.
- "중립": 객관적인 사실 전달(예: 제형, 색상 설명), 긍/부정이 혼재되어 판단이 어려운 문장. 배송, A/S 등 서비스 품질 및 제품 용기의 품질이나 외관 등, 제품 자체의 성능에 대한 판단이 아닌 문장.

{format_instructions}

# 분석할 문장:
{review_sentence}
"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI(model='gpt-4.1', temperature=0.1)
parser = JsonOutputParser(pydantic_object=Sentiment)

chain = prompt | model | parser

#### Sampling

In [ ]:
from glob import glob

file_pattern  = 'tokenized/tokenized_*.csv'
batch_size = 200
random_seed = 2025
output_file_name = 'labeled_sample.csv'

In [ ]:
csv_files = glob(file_pattern)

list_of_samples = []

for file in csv_files:
    df = pd.read_csv(file, encoding='utf-8')
    sample = df.sample(n=batch_size, random_state=random_seed)
    list_of_samples.append(sample)

In [ ]:
final_sample_df = pd.concat(list_of_samples, ignore_index=True)
print(len(final_sample_df))
final_sample_df.head()

2000


,제품명,성분,별점,피부타입,피부고민,자극도,문장
0,[보습진정] 바이오힐 보 판테셀 리페어시카 크림 70mL,"정제수, 판테놀(5%), 프로판다이올, 펜타에리스리틸테트라에틸헥사노에이트, 글리세린...",5점만점에 5점,건성에 좋아요,진정에 좋아요,자극이 느껴져요,- [ ] 저도 사용하고 많이 좋아져서 더 좋아지기를 기대하고 있습니다!
1,[10분진정] 이지듀 베리어 리페어 보습케어 크림 50ml,"정제수, 글리세린, 다이프로필렌글라이콜, 비닐다이메티콘, 카프릴릭/카프릭트라이글리세...",5점만점에 5점,건성에 좋아요,보습에 좋아요,자극없이 순해요,꾸덕하지않고 무향이라 좋아요
2,몰바니 아티초크 카밍 케어 수딩 크림 100ml,"정제수, 글리세린, 2,3-부탄다이올, 1,2-헥산다이올, 나이아신아마이드, C13...",5점만점에 5점,지성에 좋아요,진정에 좋아요,자극없이 순해요,향기부터 합격입니다!
3,[7겹보습크림] 닥터자르트 세라마이딘 울트라 모이스처라이징 크림 50ml,"정제수, 프로판다이올, 글리세린, 사이클로헥사\n실록세인, 카프릴릭/카프릭트라이글리...",5점만점에 5점,건성에 좋아요,보습에 좋아요,자극없이 순해요,닥터자르트 제품은 시카 라인이 유명하긴 한데 저는 시카가 그렇게 잘 맞는거 같지 않...
4,[7겹보습크림] 닥터자르트 세라마이딘 울트라 모이스처라이징 크림 50ml,"정제수, 프로판다이올, 글리세린, 사이클로헥사\n실록세인, 카프릴릭/카프릭트라이글리...",5점만점에 5점,복합성에 좋아요,진정에 좋아요,보통이에요,피부세라마이드와 유사한 성분이라고 하는데 그래서 피부를 더욱 건강하게 지켜주나봅니당...


In [ ]:
final_sample_df['문장']

0                - [ ] 저도 사용하고 많이 좋아져서 더 좋아지기를 기대하고 있습니다!
1                                         꾸덕하지않고 무향이라 좋아요
2                                             향기부터 합격입니다!
3       닥터자르트 제품은 시카 라인이 유명하긴 한데 저는 시카가 그렇게 잘 맞는거 같지 않...
4       피부세라마이드와 유사한 성분이라고 하는데 그래서 피부를 더욱 건강하게 지켜주나봅니당...
                              ...                        
1995        대용량 400ml짜리 써봤는데 너무 좋아서 250ml가 싸게 팔길래 재구매했습니다
1996                     예전에는 1+1세일도 많이 했는데 요즘에는 안해서 아쉽네용
1997                                 엄청 특별할것도 없이 무나ㄴ무난합니다
1998                                    만약 건성이시거나 복합성이시다!
1999    그래서 천천히 힘을 주지 않고 닦고 바른 후에 장벽 크림과 수분 앰플 꼭 사용해 줍니다!
Name: 문장, Length: 2000, dtype: object

In [ ]:
labels = []
for sentence in final_sample_df['문장']:
    try:
        response = chain.invoke(
            {
                'review_sentence':sentence, 
                'format_instructions':parser.get_format_instructions()
            }
        )
        labels.append(response['sentiment'])
    except Exception as e:
        print(f"An error occurs: {sentence} -> {e}")
        labels.append("CHAIN_ERROR")

In [ ]:
len(labels)

2000

In [ ]:
final_sample_df['label'] = labels

In [ ]:
for i in range(len(final_sample_df)):
    print(final_sample_df['label'].iloc[i],':',final_sample_df['문장'].iloc[i])

긍정 : - [ ] 저도 사용하고 많이 좋아져서 더 좋아지기를 기대하고 있습니다!
긍정 : 꾸덕하지않고 무향이라 좋아요
긍정 : 향기부터 합격입니다!
부정 : 닥터자르트 제품은 시카 라인이 유명하긴 한데 저는 시카가 그렇게 잘 맞는거 같지 않아서 선호하지 않았거덩요 ㅠㅠ
긍정 : 피부세라마이드와 유사한 성분이라고 하는데 그래서 피부를 더욱 건강하게 지켜주나봅니당 농축크림 제형이라서 겨울에 바르기 딱 죠은거 같네요
긍정 : 마무리감은 너무 무겁거나 번들거리지 않아서 주로 화장 전, 아침에 사용했습니다!
중립 : 친한 언니들에게 미리 관리 안하면 40대에 피부 컨디션 훅 간다고 관리 잘 하라는 말을 듣기도 했고 전 같지 않은 것 같기도 해서 부쩍 탄력에 관심이 가기 시작했어요
중립 : 세럼용기에 어플리케이터가 달려 있어서 바르기도 좋고 마사지도 할 수 있어요.
긍정 : 일반 아이크림 쓸때는 스파츌라로 덜어서 손가락으로
문질러 써서 번거로움이 있었는데 이제품으로 바꾸고는
너무 간편해서 좋아요
중립 : 화장 후 밀리는지는 지켜봐야겠네요
긍정 : 사이즈도 크고 맘에 듭니다.
중립 : :)
부정 : 크기는 생각보다 많이 작아요🥹
긍정 : 엄마 하나 사드렸는데 너무 좋아하시네요 ~
중립 : 🌹한달사용리뷰
수부지 피부로 수분, 영양가 둘 다 챙길 수 있는 제품을 찾다가 구매해보았습니다.

🔹
긍정 : 덕분에 아침 메이크업 전에도 부담 없이 사용할 수 있었습니다.
긍정 : 제가 생각했던 질감은 아니라서 조금 당황했는데 그래도 발림성도 좋고 트러블피부에 너무 좋아요!!
중립 : 생각이 들었어요.
긍정 : 아직 사용해본지 이틀밖에 안됐지만
뭔가 얼굴이 밝아진 느낌이랄까?ㅎㅎㅎㅎ
중립 : 토너는 화장솜에 묻혀 팩으로 자주 사용하다보니 금방금방 쓰는데 연어 크림은 용량도 짐승이예요 ㅋㅋ
중립 : 열심히 발라 볼게요~~
부정 : 수딩 크림이지만 다른 제품처럼 많이 바르면 오히려 다음날 여ㄷ름이 올라와서 소량으로 얇게 펴서 바르고 있어요
중립 : * 사용자 정보.
중립 : 가을 겨울철에

In [ ]:
final_sample_df.to_csv(output_file_name, index=False, encoding='utf-8')